In [1]:
# get workshop code
import os
import sys
IN_COLAB = os.getenv("COLAB_RELEASE_TAG")
if IN_COLAB:
    !git clone https://github.com/rajaonsonella/crosstalk-q2-2025
    sys.path.append('./crosstalk-q2-2025')
else:
    sys.path.append('..')
!pip install -r crosstalk-q2-2025/requirements.txt

Cloning into 'crosstalk-q2-2025'...
remote: Enumerating objects: 369, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 369 (delta 74), reused 41 (delta 27), pack-reused 248 (from 1)
Receiving objects: 100% (369/369), 37.60 MiB | 39.33 MiB/s, done.
Resolving deltas: 100% (198/198), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 61.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# Download data from google drive
import gdown
import os

file_ids = {'test_inputs' : '1Gyv_ldUTi0Ymy6wVMfruAO0UraCQ70CR',
            'train': '11S5p0QgP1X9rOFiIjNSLydLenJwm7hle'}

for name, file_id in file_ids.items():
    filename = f'crosstalk_{name}.parquet'
    if not os.path.exists(filename):
        gdown.download(id=file_id, output=filename, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Gyv_ldUTi0Ymy6wVMfruAO0UraCQ70CR
From (redirected): https://drive.google.com/uc?id=1Gyv_ldUTi0Ymy6wVMfruAO0UraCQ70CR&confirm=t&uuid=ee8140ca-a6f9-4a8f-a585-1f3a9c92dad7
To: /content/crosstalk_test_inputs.parquet
100%|██████████| 1.52G/1.52G [00:11<00:00, 137MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=11S5p0QgP1X9rOFiIjNSLydLenJwm7hle
From (redirected): https://drive.google.com/uc?id=11S5p0QgP1X9rOFiIjNSLydLenJwm7hle&confirm=t&uuid=21c76152-cacd-4eff-8862-bda31ccc3d4e
To: /content/crosstalk_train.parquet
100%|██████████| 1.97G/1.97G [00:19<00:00, 102MB/s]


In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 22.4 MB/s eta 0:00:00


## Train with ECFP6

In [5]:
# === Imports ===
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from eval import BinaryEvaluator  # assumes you have this module

# === Optional Balanced Data Loader ===
def load_balanced_dataframe(path, x_col, y_col, ratio=2, balance=True):
    """
    Load a DataFrame from parquet and optionally balance it to a given negative:positive ratio.
    """
    df = pd.read_parquet(path)

    if not balance:
        return df

    pos = df[df[y_col] == 1]
    neg = df[df[y_col] == 0].sample(n=len(pos) * ratio, random_state=42)
    df_balanced = pd.concat([pos, neg]).sample(frac=1, random_state=42)  # shuffle

    return df_balanced

# === Load and Preprocess Data ===
balance_data = True  # Toggle this flag to use balanced or full dataset
data_path = "/content/crosstalk_train.parquet"
x_col = "ECFP6"
y_col = "DELLabel"

df_train = load_balanced_dataframe(path=data_path, x_col=x_col, y_col=y_col, ratio=2, balance=balance_data)
X_full = df_train[x_col].str.split(',', expand=True).astype(float).values
y_full = df_train[y_col].values

# === Optuna Objective Function ===
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 15, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),  # min_samples_leaf
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),  # L1
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),  # L2
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
        "max_bin": trial.suggest_int("max_bin", 127, 511),
        "boosting_type": "gbdt",
        "n_jobs": 1,
        "use_best_model": True,
        "random_state": 42,
    }

    auc_scores = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X_full, y_full):
        X_train, X_val = X_full[train_idx], X_full[val_idx]
        y_train, y_val = y_full[train_idx], y_full[val_idx]

        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)
        auc_scores.append(auc)

    return np.mean(auc_scores)

# === Run Optuna ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Increase for better results

# === Best Model Info ===
print("\nBest Trial for AUROC (5-fold CV):")
print(f"Mean AUROC: {study.best_value:.4f}")
print("Best Parameters:")
for key, val in study.best_params.items():
    print(f"  {key}: {val}")

# === Final Model Training ===
best_params = study.best_params
best_params.update({
    "boosting_type": "gbdt",
    "n_jobs": 1,
    "use_best_model": True,
    "random_state": 42
})

final_model = LGBMClassifier(**best_params)
final_model.fit(X_full, y_full)

# === Final Evaluation ===
evaluator = BinaryEvaluator(X_full, y_full)
metric_dict_cv = evaluator.CV_model(final_model)

print("\n📊 Final LightGBM Model Evaluation (CV):")
for metric_name, metric_value in metric_dict_cv['mean'].items():
    print(f'{metric_name:20s}: {metric_value:.4f}')


[I 2025-06-26 01:16:04,955] A new study created in memory with name: no-name-0872a77b-4955-46f4-a004-25da69f316e8
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Info] Number of positive: 23023, number of negative: 46044
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.510368 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7161
[LightGBM] [Info] Number of data points in the train set: 69067, number of used features: 2048
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333343 -> initscore=-0.693104
[LightGBM] [Info] Start training from score -0.693104
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: use_best_model


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Info] Number of positive: 23022, number of negative: 46045
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.696154 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7177
[LightGBM] [Info] Number of data points in the train set: 69067, number of used features: 2048
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333329 -> initscore=-0.693169
[LightGBM] [Info] Start training from score -0.693169
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Info] Number of positive: 23023, number of negative: 46045
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.703059 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7181
[LightGBM] [Info] Number of data points in the train set: 69068, number of used features: 2048
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333338 -> initscore=-0.693125
[LightGBM] [Info] Start training from score -0.693125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: use_best_model

📊 Final LightGBM Model Evaluation (CV):
accuracy            : 0.9348
balanced_accuracy   : 0.9240
roc_auc             : 0.9794
precision           : 0.9110
recall              : 0.8915
mrr                 : 0.0016
precision_at_k_5    : 1.0000
hits_at_k_5         : 0.0009
precision_at_k_10   : 1.0000
hits_at_k_10        : 0.0017
precision_at_k_30   : 1.0000
hits_at_k_30        : 0.0052


## Train with AutoGluon

In [6]:
from dataset import basic_dataloader
team_name = 'nky'

In [7]:
%%time
X_test = basic_dataloader('/content/crosstalk_test_inputs.parquet', x_col="ECFP6", y_col = None, max_to_load = None, chunk_size = 20000)

Loading chunks:   0%|          | 0/17 [00:00<?, ?it/s]

CPU times: user 2min 51s, sys: 9.94 s, total: 3min 1s
Wall time: 2min 59s


In [8]:
yp = final_model.predict_proba(X_test)[:,1]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: use_best_model


In [9]:
import pyarrow.parquet as pq
pf = pq.ParquetFile('/content/crosstalk_test_inputs.parquet')

In [10]:
pf.metadata

  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 12
  num_rows: 339258
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 59684

In [11]:
preds = pf.read(columns=['RandomID']).to_pandas()
preds['DELLabel'] = yp
display(preds)

,RandomID,DELLabel
0,ID_0,0.045122
1,ID_1,0.017088
2,ID_2,0.000332
3,ID_3,0.005461
4,ID_4,0.022786
...,...,...
339253,ID_339253,0.000516
339254,ID_339254,0.000301
339255,ID_339255,0.003515
339256,ID_339256,0.019079


In [ ]:
preds.to_csv(f'{team_name}.csv', index=False)